In [ ]:
import cv2
import numpy as np

import imutils

from keras.models import load_model

In [ ]:
model = load_model("harryTest.h5") # load the trained model 

cap = cv2.VideoCapture(0)

while(True):
    ret, frame = cap.read()
    # preprocess the image by converting it to grayscale, binary image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray.copy(), 127, 255, cv2.THRESH_BINARY_INV)[1]
    # find contours in the thresh, then sort them by their size in descending order
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse = True)
    digitCnts = cnts[0:4] # the first 5 largest contours
    for c in digitCnts:
        (x, y, w, h) = cv2.boundingRect(c) # bounding rectangles

        # extract the digit ROI from thresh (binary)
        roi = thresh[y:y+h, x:x+w]
        d = max(h, w)
    
        square_roi = np.zeros((d, d))
    
        # square roi
        if h == d:
            square_roi[0:h, int(h/2 - w/2):int(h/2 + w/2)] = roi # broadcast the roi into the middle the square image
        
        elif w == d:
            square_roi[int(w/2 - h/2):int(w/2 + h/2), 0:w] = roi
    
        # resize the squared roi
        resize_roi = cv2.resize(square_roi, (28, 28))
    
        # reshape the 28 by 28 roi to match the requirements in the model
        test_image = np.reshape(resize_roi, [1, 28, 28, 1])
    
        # predict the digit
        predict_digit = np.argmax(model.predict(test_image), axis=-1)
    
        # draw the rectangle around the digits
        cv2.rectangle(frame,(x,y), (x+w, y+h), (255, 255, 0), 10)
    
        # show the predicted digits on the image
        cv2.putText(frame, str(predict_digit), (x-30, y-70),
               cv2.FONT_HERSHEY_SIMPLEX, 4, (255, 255, 0), 10)
        cv2.imshow("d", frame)
        
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
